In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import (datetime as dt, timedelta)
#import datetime
import pytrends
from pytrends.request import TrendReq

### load data

In [ ]:
#data_path = "../data/interim" 
knit_data = pd.read_csv("../data/interim/transactions_sd_knits_resampled.csv")

In [ ]:
knit_data['transaction_date'] = pd.to_datetime(knit_data['transaction_date'], infer_datetime_format=True)

### engineer columns

In [ ]:
def engineer_columns(df):
    '''
    Attaches engineered columns including 'week_no', 'month', 'price' per week, 'price_comp_week' and 'transaction_date' for the first Sunday of a week.
        Parameters:
            df (df): Dataframe to engineer and attach columns to.
        Returns:
            df (df): Dataframe with engineered columns attached.
    '''
    # sort data
    df = df.sort_values(by=['p_id', 'transaction_date'])

    # add week no
    df['week_no'] = df['transaction_date'].dt.strftime('%U')

    # concatenate by week and sum quantity 
    knit_data_week = df.groupby(['p_id', 'week_no', 'sub_department_desc', 'label_desc', 'color_simple'])['quantity'].sum().to_frame('quantity').reset_index()

    # Calculate price as average of amount per week / quantity
    sum_amount_week = df.groupby(['p_id', 'week_no', 'sub_department_desc', 'label_desc', 'color_simple'])['amount'].sum().to_frame('sum_amount').reset_index()
    knit_data_week = pd.merge(knit_data_week, sum_amount_week, on=['p_id', 'week_no', 'sub_department_desc', 'label_desc','color_simple'], how='left')
    knit_data_week['price'] = knit_data_week['sum_amount']/knit_data_week['quantity']
    knit_data_week.drop(columns=['sum_amount'], inplace=True)
    df = knit_data_week

    # engineer price competition column per week 
    price_mean_week = df.groupby(['week_no'])['price'].mean().to_frame("mean_price_week").reset_index()
    df = pd.merge(df, price_mean_week, on='week_no', how='left')
    df['price_comp_week'] = df['price']/df['mean_price_week']
    df.drop(columns=['mean_price_week'], inplace=True)

    # add date for first sunday back in
    def find_sunday(week):
        ref = '2021-01-03' #reference date corresponding to the 1st Sunday in 2021
        ref_object = dt.strptime(ref,'%Y-%m-%d') #reference day converted in datetime object
        day_object = ref_object + timedelta(days=(week-1)*7) # adding the number of days
        day=day_object.strftime('%Y-%m-%d') # converting back to the desired format
        return day

    df['transaction_date'] = df['week_no'].apply(lambda x : find_sunday(int(x)))

    # add month column
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], infer_datetime_format=True)
    df['month'] = df['transaction_date'].dt.strftime('%b')

    df = df.sort_values(by=['p_id', 'transaction_date'])

    return df


In [ ]:
knit_data = engineer_columns(knit_data)

### define function - star rating and sentiment

In [ ]:
def synthesise_star_sentiment(df, random_seed=123, star_dist = [0.03675, 0.06773, 0.12719, 0.23374, 0.53459]):
    '''
    Returns two synthetic features i) star_rating (1-5) randomly assigned based on the distribution profile 
    of star ratings given to women's knitwear in the following dataset:
    https://www.kaggle.com/datasets/nicapotato/womens-ecommerce-clothing-reviews?datasetId=11827&sortBy=voteCount
    Future star rating could be assigned to a transaction based on the average rating of an item at the time of transaction.
    ii) review sentiment, -1 for negative, 0 for neutral and 1 for positive, assigned to transaction based on
    star rating. Distribution of assignment was based on educated guess. In future actual reviews could be analysed
    by Natural language processing to determine the average review sentiment of an item at the time of tranaction.
        
            Parameters:
                df (dataframe): Dataframe to append new features of star rating and review sentiment
                random_seed (int): Random number (default is 123)
                star_dist (list): List of floats which represent the distribution profile
            Returns:
                df (dataframe): Dataframe with appended columns 'star_rating' for assigned star rating and                
                 'review' to capture review sentiment 
    '''
    
    np.random.seed(random_seed)

    df['star_rating'] = np.nan

    df['star_rating'] = np.random.choice([1, 2, 3, 4, 5], size=df.shape[0], replace=True, p=star_dist) 
    
    mask1 = df['star_rating'] >= 4
    mask2 = (df['star_rating'] < 4) & (df['star_rating'] >= 2)
    mask3 = df['star_rating'] < 2

    df.loc[mask1, 'review'] = 1
    df.loc[mask2, 'review'] = np.random.choice([-1, 0, 1], replace = True, p=[.2, .65, .15])
    df.loc[mask3, 'review'] = np.random.choice([-1, 0, 1], replace = True, p=[.9, .08, .02])

    df['review'] = df['review'].astype(str)

    return df

#star_dist_sales = [0, 0.07, 0.14, 0.24, 0.55]
#star_dist_returns = [0.18, 0.2, 0.26, 0.36, 0])


### synthesise data

In [ ]:
knit_data = synthesise_star_sentiment(df=knit_data)

In [ ]:
knit_data['star_rating'].hist()

In [ ]:
knit_data['review'].hist()

### Functions to download google trends data and convert to interest rate of change

In [ ]:
search_list = ['Black Knit', 'White Knit', 'Zebra Knit', 'Blue Knit', 'Green Knit',
            'Pink Knit', 'Yellow Knit', 'Cream Knit', 'Brown Knit']

search_start = '2020-12-27 ' 
search_end = '2021-12-31'

def get_google_trends(search_list, search_start, search_end):
    '''
    Returns new dataframe of google trend's weekly interest over time of a search term in search list. 
    Values are relative to the highest search volume of that term in the defined time period in a given 
    georgraphical region (here GB). A value of 100 is the peak popularity for the term, 50 means that 
    the term is half as popular. A score of 0 means that there was not enough data for this term. Designed 
    for function to be run for each new google_trend feature to be added to the dataframe, which is paired 
    to another feature in the dataframe, with a search list containing the categorical column options.  
    
            Parameters:
                search_list (list of str): Terms to be searched - corresponds to different categorical 
                search_start (str): date in 'YYYY-MM-DD' to avoid NaN downstream date should be two weeks before 
                                    start of corresponding transactions dataframe
                search_end (str): date in 'YYYY-MM-DD'

            Returns:
                dataframe of all search terms with google ternds relative interest over specified time period
    '''

    pytrend = TrendReq(hl='en-UK', tz=0)
    trends_dict = {}
    df_trends = pd.DataFrame()

    for term in search_list:
        pytrend.build_payload(
        kw_list=[term],
        cat=0,
        geo='GB-ENG',
        timeframe=(search_start+search_end))
        trends_dict[term] = pytrend.interest_over_time()

    df_trends = pd.concat([trends_dict[key] for key in search_list], join = 'inner', axis =1)
    return df_trends.drop(labels=['isPartial'],axis='columns')

In [ ]:
# Google trends interest relative to the first week of 2021 
def calculate_google_trend_term_relative_to_week_1(df):    
    '''
    Returns google trend dataframe with interest over time relative to the interest in first week in 2021

            Parameters:
                df (dataframe): google trends dataframe

            Returns:
                df (dataframe): google trends dataframe with values relative to week 1 2021

    ''' 
    return df.div(df.iloc[1]) # Week1 2021 is not first row of this dataframe hence position [1]
            

In [ ]:
def calculate_rate_of_change_of_google_trend_term(df):
    '''
    Function to compare current google trend information to the 3 week moving average as a rate of change 
    Negative values imply term is searched less than the 3 week moving average, Positive values, more than.
    NB: Not used in the end for the project - maybe useful for an ARIMA model?

            Parameters:
                df (dataframe): google trends dataframe

            Returns:
                df_diff_MA (dataframe): google trends dataframe with interest values relative to 3 week moving average
    '''
    
    moving_dict = {}

    for col in df.columns:
        moving_dict[col] = df[col].rolling(3, min_periods=3).mean()
    df_moving_average = pd.concat([moving_dict[col] for col in df.columns], join = 'inner', axis=1)   

    df_diff_MA = df.subtract(df_moving_average, axis=1)

    return df_diff_MA.div(df_moving_average, axis=1)

In [ ]:
# Function to expand out trends from week to days 
def make_trends_daily(df, search_start, search_end):
    '''
    Returns expanded google trends dataframe on daily rather than weekly basis 
    (forward fill of sunday value)

            Parameters:
                df (dataframe): google trends dataframe with values relative to week 1 2021

            Returns:
                df (dataframe): daily google trends dataframe with values relative to week 1 2021 

    '''
    
    date_range = pd.date_range(start=search_start, end=search_end)
    return df.reindex(date_range).fillna(method='ffill')

In [ ]:
# Google trend feature for colour of knit
trend_df = get_google_trends(search_list, search_start, search_end)  
colour_trend = calculate_google_trend_term_relative_to_week_1(trend_df)
colour_trend_daily = make_trends_daily(colour_trend, search_start, search_end)

In [ ]:
# Google trend feature relating to style of 'knits'
style_knits = get_google_trends(['Knits'], search_start, search_end)   
trend_knits = calculate_google_trend_term_relative_to_week_1(style_knits) 
knits_trend_daily = make_trends_daily(trend_knits, search_start, search_end)


### Convert to feature on the transactions dataframe 

In [ ]:
#knits_transactions = pd.read_csv("../data/interim/transactions_sd_knits_resampled_synth.csv") ### aminah

In [ ]:
#knits_transactions['transaction_date'] = knits_transactions['transaction_date'].apply(pd.to_datetime) ### aminah

### Function to add sub department knits google trend
TOFIX: This will not work if more than one sub department is presented to be searched for in the google trends

In [ ]:
def append_google_trends_sub_depart_feature(df_transactions, df_google_trends):
    '''
    Returns column appended to transaction data frame with the google trend interest of the subdepartment 
    term ie 'knits' based on the date of transaction. NB this for use of appending google trends
    data where there is only one search term otherwise use function append_google_trends_colour_feature. 

            Parameters:
                df_transactions (dataframe): dataframe with tranactions and physical attributes of items
                df_google_trends (dataframe): daily google trends dataframe with values relative to week 1 2021 

                
            Returns:
                df_trans_gt_knit (dataframe): transactions dataframe with appended 'google_trend_knit' variable

    '''
    
    df_google_trends.reset_index(inplace=True)

    df_trans_gt_knit = df_transactions.merge(df_google_trends, 
                                            left_on='transaction_date', 
                                            right_on='index', 
                                            how='left')
    
    df_trans_gt_knit.drop(columns=['index'], inplace=True)  
    df_trans_gt_knit.rename(columns={'Knits':'google_trends_knit'}, inplace=True)
    
    return df_trans_gt_knit

In [ ]:
knits_transactions_gt_knits = append_google_trends_sub_depart_feature(knit_data, knits_trend_daily) ### aminah

In [ ]:
def append_google_trends_colour_feature(df_transactions, df_google_trends):
    '''
    Returns transaction dataframe with the appropriate google_trends value based on the colour in 'color_simple' 
    and the transaction date for each transaction. 

            Parameters:
                df_transactions (dataframe): dataframe with tranactions and physical attributes of items
                df_google_trends (dataframe): daily google trends dataframe with values relative to week 1 2021 

            Returns:
                df_transactions (dataframe): transactions dataframe with appended 'google_trend_colour' variable

    '''
    
    df_transactions['google_trends_colour'] = np.nan

    df_google_trends.columns = df_google_trends.columns.str.replace(' Knit', '')

    for i, row in df_transactions.iterrows():

        # get date and color in transactions
        transaction_date = f"{row['transaction_date']}"
        x = f"{row['color_simple']}"

        if x != 'Other':
        # get correct colour column
            gt_colour = df_google_trends[df_google_trends.columns[df_google_trends.columns.isin([x])]]
            gt_colour.reset_index(inplace=True)

            mask = gt_colour['index'] == transaction_date

            gt = gt_colour.loc[mask, x]

            gt = gt.values
       
            df_transactions['google_trends_colour'].iloc[i] = gt
        else:
            df_transactions['google_trends_colour'].iloc[i] = 0 # No change in other trend

    return df_transactions

In [ ]:
append_google_trends_colour_feature(knits_transactions_gt_knits, colour_trend_daily)

In [ ]:
knit_data = knits_transactions_gt_knits

### drop knit column redundant

In [ ]:
knit_data.drop(columns=['sub_department_desc'], inplace=True)

### rearrange columns

In [ ]:
knit_data = knit_data[['p_id','transaction_date', 'week_no', 'month', 'label_desc', 'color_simple', 'quantity', 
        'price', 'price_comp_week', 'star_rating', 'review', 'google_trends_knit', 'google_trends_colour']]

### Sort on date 

In [ ]:
knit_data = knit_data.sort_values(by=['transaction_date'])

### write to interim folder

In [ ]:
knit_data.to_csv("../data/interim/transactions_sd_knits_resampled_engin_synth_gt.csv", index=False)

In [ ]:
knit_data.info()